In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 56 * 56)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def load_data():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    data_transform = datasets.ImageFolder(root='/content/drive/MyDrive/Pneumonia/X-Ray', transform=transform)

    train_size = int(0.75 * len(data_transform))
    val_size = int(0.15 * len(data_transform))
    test_size = len(data_transform) - train_size - val_size

    train_data, val_data, test_data = torch.utils.data.random_split(data_transform, [train_size, val_size, test_size])

    extra_val_data, train_data = torch.utils.data.random_split(train_data, [int(0.1 * len(train_data)), len(train_data) - int(0.1 * len(train_data))])

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)
    test_loader = DataLoader(test_data, batch_size=32)

    return train_loader, val_loader, test_loader


In [ ]:
def evaluate_test_set(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    test_accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    confusion = confusion_matrix(true_labels, predicted_labels)
    classification_rep = classification_report(true_labels, predicted_labels)

    return test_accuracy, precision, recall, f1, confusion, classification_rep


In [ ]:
# Veri setini yükleme ve CNN modelinin eğitimi
train_loader, val_loader, test_loader = load_data()
cnn_model = CNN()
train_evaluate_cnn(cnn_model, train_loader, val_loader, test_loader)


In [ ]:
# Test seti üzerinde modelin performansını değerlendirme
test_accuracy, precision, recall, f1, confusion, classification_rep = evaluate_test_set(cnn_model, test_loader)

print(f"Test Accuracy: {test_accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")